In [1]:
#import Pkg; Pkg.add("Manifolds")

In [2]:
using Manifolds
M = Sphere(2)
γ = shortest_geodesic(M, [0., 0., 1.], [0., 1., 0.])
γ(0.5)

3-element Vector{Float64}:
 0.0
 0.7071067811865475
 0.7071067811865476

In [3]:
#Pkg.add("ManifoldsBase")

In [4]:
using ManifoldsBase, LinearAlgebra, Test
import ManifoldsBase: check_manifold_point, check_tangent_vector, manifold_dimension, exp!

By the way, typing `\bbR` is the way to get $\mathbb{R}$ in julia code.

In [5]:
using LinearAlgebra, ForwardDiff

"""
    AlgebraicSet <: Manifold{ℝ}

Define an algebraic set. Construct by `AlgebraicSet(gs,d,N,tol)`
where `gs` is a list of polynomial functions whose zeros define the algebraic set
and `dim` is a nonnegative integer, the dimension of the algebraic set.
Then `N` is the ambient dimension, which is the number of variables in the polynomials.
Finally `tol` is the tolerance for checking if a point is in the algebraic set. If we
evaluate the polynomial functions at a point, and the norm of the resulting vector of nearly zero
entries is less than `tol`, then we judge the point to be on the algebraic set.
"""

struct AlgebraicSet <: Manifold{ManifoldsBase.ℝ}
    eqns
    varietydim::Int
    ambientdim::Int
    numeqns::Int
    residualtol::Float64
    f
    df
    f!
end

function AlgebraicSet(eqns,d::Int,N::Int,tol::Float64)
    k = length(eqns)
    if k==1
        f = x -> eqns[1](x)
        df = x -> ForwardDiff.gradient(eqns[1], x)
    else
        f = x -> [eqn(x) for eqn in eqns]
        df = x -> ForwardDiff.jacobian(f, x)
    end
    f! = (F,x) -> begin
        for i in 1:k
            F[i] = eqns[i](x)
        end
        for i in (k+1):N
            F[i] = 0.
        end
        return F
    end
    return AlgebraicSet(eqns,d,N,k,tol,f,df,f!)
end
AlgebraicSet(eqns,d::Int,N::Int) = AlgebraicSet(eqns,d,N,1e-8) # default tolerance

Base.show(io::IO, M::AlgebraicSet) = print(io,
    "An algebraic set of dimension $(M.varietydim) with ambient dimension $(
    M.ambientdim) defined by the $(M.numeqns) polynomials $(M.eqns).")

In [6]:
g1(x) = (x[1]^4 + x[2]^4 - 1) * (x[1]^2 + x[2]^2 - 2) + x[1]^5 * x[2]
gs = [g1]
dim = 1
ambientdim = 2

M = AlgebraicSet(gs,dim,ambientdim)

An algebraic set of dimension 1 with ambient dimension 2 defined by the 1 polynomials [g1].

In [7]:
p = [1.0; 0.0] # g1(p) = 0, so p is a point on the variety V(g1)
X = [1.0; 4.0] # check if this is a tangent vector, yes!

M.f(p), M.df(p)

(0.0, [-4.0, 1.0])

In [8]:
M.df(p)'*X # dot product is zero since v is a tangent vector to p

0.0

In [9]:
function check_manifold_point(M::AlgebraicSet, p)
    # p is a point on the manifold
    (size(p)) == (M.ambientdim,) || return DomainError(size(p),"The size of $p is not $(M.ambientdim).")
    if norm( [eqn(p) for eqn in M.eqns] ) > M.residualtol
        return DomainError(p,
            "The norm of vector of evaluations of the equations at $p is not less than $(M.residualtol).")
    end
    return nothing
end

function check_tangent_vector(M::AlgebraicSet, p, X, check_base_point = true)
    # p is a point on the manifold, X is a tangent vector
    if check_base_point
        mpe = check_manifold_point(M, p)
        mpe === nothing || return mpe
    end
    size(X) != size(p) && return DomainError(size(X), "The size of $X is not $(size(p)).")
    if M.numeqns == 1
        if M.df(p)' * X > M.residualtol
            return DomainError( M.df(p)' * X, "The tangent $X is not orthogonal to $p.")
        end
    else
        if norm(M.df(p) * X) > M.residualtol
            return DomainError( norm(M.df(p) * X), "The tangent $X is not orthogonal to $p.")
        end
    end
    return nothing
end;

In [10]:
is_manifold_point(M, randn(2)) # should be false

false

In [11]:
@test_throws DomainError is_manifold_point(M, rand(3), true) # only on R^2, throws an error.

Test Passed
      Thrown: DomainError

In [12]:
# The following two tests return true
[ is_manifold_point(M, p); is_tangent_vector(M,p,X) ]

2-element Vector{Bool}:
 1
 1

In [13]:
manifold_dimension(M::AlgebraicSet) = M.varietydim

manifold_dimension(M)

1

In [14]:
using NLsolve

function exp!(M::AlgebraicSet, q, p, X)
    # mutates `q` to refer to the point on the manifold in tangent direction `X` from point `p`
    nX = norm(X)
    if nX == 0
        q .= p
    else
        #q .= cos(nX/M.radius)*p + M.radius*sin(nX/M.radius) .* (X./nX)
        initpt = p + X
        result = NLsolve.nlsolve(M.f!, initpt, autodiff=:forward)
        #println(result)
        q .= result.zero
    end
    return q
end

exp! (generic function with 33 methods)

In [15]:
M.f!

#6 (generic function with 1 method)

In [16]:
p = [1.0; 0.0] # g1(p) = 0, so p is a point on the variety V(g1)
X = [1.0; 4.0] # check if this is a tangent vector, yes!

X = normalize(X) / 100.

2-element Vector{Float64}:
 0.0024253562503633295
 0.009701425001453318

In [17]:
q = exp(M, p, X) # takes a moment because we're using NLsolve for the first time...

2-element Vector{Float64}:
 1.0024565557611786
 0.009693409861657605

In [18]:
is_manifold_point(M,q)

true

In [19]:
q ∈ M

true

In [20]:
#import Pkg; Pkg.add("Manopt")

In [ ]:
#using Manifolds, Manopt
#random_tangent(M, p, Val(:Gaussian))

# the kernel dies!

Below is an example of using `NLsolve.jl` to compute solutions to nonlinear equations.

Figuring out `ForwardDiff` package...

In [34]:
using ForwardDiff

p = [1.0; 0.0] # g1(p) = 0, so p is a point on the variety V(g1)
#v = [3.0; 4.0] # check if this is a tangent vector, nope.
v = [1.0; 4.0] # check if this is a tangent vector, yes!
v = normalize(v)
n1 = ForwardDiff.gradient(g1, p)
n1'v # not zero, so v is not a tangent vector at p

0.0

In [32]:
f1(x) = x[1]^2 + x[2]^2 + x[3]^2 - 1
f2(x) = x[1] - 0.5

f(x) = [f1(x), f2(x)]

df = x -> ForwardDiff.jacobian(f, x)

p = [0.5, sqrt(3/4), 0.0] #f(p) gives approximately zero
df(p)

2×3 Matrix{Float64}:
 1.0  1.73205  0.0
 1.0  0.0      0.0